In [8]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np

# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
#app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
    dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
    dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
    dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3'),
    html.Div(id='page-content')
])

# Leer la imagen y codificarla en base64
with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'

@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':
        return html.Div([
                html.H3('Página 1',style={'background-color': '#0000FF', 'color': 'white', 'padding': '10px'}),
                html.Div([
                    dcc.DatePickerRange(
                            id='date-picker-range',
                            start_date=df.index.min(),
                            end_date=df.index.max()
                        ),
                        dcc.Dropdown(  # Agregar selección de agregación de tiempo
                            id='time-aggregation',
                            options=[
                                {'label': 'Horaria', 'value': 'H'},
                                {'label': 'Diaria', 'value': 'D'},
                                {'label': 'Semanal', 'value': 'W'},
                                {'label': 'Mensual', 'value': 'M'},
                                {'label': 'Anual', 'value': 'Y'}
                            ],
                            value='H'
                        ),

                    html.Div([
                    html.H6(["Birst Level Metrics"], className="gs-header gs-text-header padded",style={'marginTop': 15})
                    ]),

            ]), dbc.Row(id='graph-row', children=[])

            ])
                    
    
    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data, style={'width':'20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('graph-row', 'children'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date'),
    Input('time-aggregation', 'value')
)
def update_graphs(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig2 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6)
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    Input('date-picker-range-page2', 'start_date'),
    Input('date-picker-range-page2', 'end_date'),
    Input('time-aggregation-page2', 'value')
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]

@app.callback(
    Output('graph-row-page3', 'children'),
    Input('date-picker-range-page3', 'start_date'),
    Input('date-picker-range-page3', 'end_date'),
    Input('time-aggregation-page3', 'value')
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np

# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
    dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
    dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
    dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3'),
    html.Div(id='page-content')
])

# Leer la imagen y codificarla en base64
with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'

@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    data = {
    "Predicción": ["Predicción"],
    **{f"H{i}": [i] for i in range(1, 25)},
    "Media": [25]
    }
    df = pd.DataFrame(data)
    if pathname == '/page-1':
        return html.Div([
        html.H3('Meteológica', style={'background-color': '#2ECC71', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}),
        html.Div([
            dcc.DatePickerRange(
                id='date-picker-range',
                start_date=df.index.min(),
                end_date=df.index.max(),
                style={'display': 'inline-block', 'vertical-align': 'middle', 'marginRight': '10px'}
            ),
            dcc.Dropdown(
                id='time-aggregation',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H',
                style={'width': '150px', 'display': 'inline-block', 'vertical-align': 'middle'}
            ),
            html.Div([
                html.H6(["Birst Level Metrics"], className="gs-header gs-text-header padded",
                        style={'marginTop': 15}), 
                        html.Div([dash_table.DataTable(
                                    data=df.to_dict('records'),
                                    columns=[{"name": col, "id": col} for col in df.columns],
                                    style_header={
                                        "backgroundColor": "#4863A0",
                                        "color": "white",
                                        "fontWeight": "bold",
                                        "textAlign": "center"
                                    },
                                    style_cell={
                                        "textAlign": "center"
                                    }
                                )])
            ]),
        ]),
        dbc.Row(id='graph-row', children=[])
    ])

    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data,
                     style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('graph-row', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation', 'value')]
)
def update_graphs(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig2 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6)
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    [Input('date-picker-range-page2', 'start_date'),
     Input('date-picker-range-page2', 'end_date'),
     Input('time-aggregation-page2', 'value')]
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]

@app.callback(
    Output('graph-row-page3', 'children'),
    [Input('date-picker-range-page3', 'start_date'),
     Input('date-picker-range-page3', 'end_date'),
     Input('time-aggregation-page3', 'value')]
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np
from dash import dash_table
from dash.dash_table.Format import Group


# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
    dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
    dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
    dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3'),
    html.Div(id='page-content')
])

# Leer la imagen y codificarla en base64
with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'

@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':
        return html.Div([
            html.H3('Meteológica', style={'background-color': '#2ECC71', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}),
            html.Div([
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=df.index.min(),
                    end_date=df.index.max(),
                    style={'display': 'inline-block', 'vertical-align': 'middle', 'marginRight': '10px'}
                ),
                dcc.Dropdown(
                    id='time-aggregation',
                    options=[
                        {'label': 'Horaria', 'value': 'H'},
                        {'label': 'Diaria', 'value': 'D'},
                        {'label': 'Semanal', 'value': 'W'},
                        {'label': 'Mensual', 'value': 'M'},
                        {'label': 'Anual', 'value': 'Y'}
                    ],
                    value='H',
                    style={'width': '150px', 'display': 'inline-block', 'vertical-align': 'middle'}
                ),
                html.Div(id='birst-level-metrics')
            ]),
            dbc.Row(id='graph-row', children=[])
        ])
    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data,
                     style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('birst-level-metrics', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_birst_level_metrics(start_date, end_date):
    filtered_df = df.loc[start_date:end_date]
    last_24_records = filtered_df['Y'].tail(24)

    data = {
        "Hora": ["Predicción"],
        **{f"H{i}": [value] for i, value in enumerate(last_24_records, start=1)},
        "Media": [last_24_records.mean()]
    }
    df_table = pd.DataFrame(data)
    table = dash_table.DataTable(
        data=df_table.to_dict('records'),
        columns=[{"name": col, "id": col} for col in df_table.columns],
        style_header={
            "backgroundColor": "#4863A0",
            "color": "white",
            "fontWeight": "bold",
            "textAlign": "center"
        },
        style_cell={
            "textAlign": "center"
        }
    )
    current_date = datetime.now().date().strftime("%Y-%m-%d")
    return [html.H6([f"Predicción Day-Ahead {current_date}"], className="gs-header gs-text-header padded"), table]


@app.callback(
    Output('graph-row', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation', 'value')]
)
def update_graphs(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig2 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6)
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    [Input('date-picker-range-page2', 'start_date'),
     Input('date-picker-range-page2', 'end_date'),
     Input('time-aggregation-page2', 'value')]
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]

@app.callback(
    Output('graph-row-page3', 'children'),
    [Input('date-picker-range-page3', 'start_date'),
     Input('date-picker-range-page3', 'end_date'),
     Input('time-aggregation-page3', 'value')]
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np
from dash_table.Format import Group
from dash import dash_table


# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
    dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
    dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
    dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3'),
    html.Div(id='page-content')
])

# Leer la imagen y codificarla en base64
with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'

@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':
        return html.Div([
            html.H3('Meteológica', style={'background-color': '#2ECC71', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}),
            html.Div([
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=df.index.min(),
                    end_date=df.index.max(),
                    style={'display': 'inline-block', 'vertical-align': 'middle', 'marginRight': '10px'}
                ),
                dcc.Dropdown(
                    id='time-aggregation',
                    options=[
                        {'label': 'Horaria', 'value': 'H'},
                        {'label': 'Diaria', 'value': 'D'},
                        {'label': 'Semanal', 'value': 'W'},
                        {'label': 'Mensual', 'value': 'M'},
                        {'label': 'Anual', 'value': 'Y'}
                    ],
                    value='H',
                    style={'width': '150px', 'display': 'inline-block', 'vertical-align': 'middle'}
                ),
                html.Div(id='birst-level-metrics')
            ]),
            dbc.Row(id='graph-row', children=[])
        ])
    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data,
                     style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('birst-level-metrics', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_birst_level_metrics(start_date, end_date):
    filtered_df = df.loc[start_date:end_date]
    last_24_records = filtered_df['Y'].tail(24)

    data = {
        "Hora": ["Predicción"],
        **{f"H{i}": [value] for i, value in enumerate(last_24_records, start=1)},
        "Media": [last_24_records.mean()]
    }
    df_table = pd.DataFrame(data)
    table = dash_table.DataTable(
        data=df_table.to_dict('records'),
        columns=[{"name": col, "id": col} for col in df_table.columns],
        style_header={
            "backgroundColor": "#4863A0",
            "color": "white",
            "fontWeight": "bold",
            "textAlign": "center"
        },
        style_cell={
            "textAlign": "center"
        }
    )
    current_date = datetime.now().date().strftime("%Y-%m-%d")
    return [html.H6(f"Predicción Day-Ahead {current_date}", className="gs-header gs-text-header padded"), table]


@app.callback(
    Output('graph-row', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation', 'value')]
)
def update_graphs(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig2 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6)
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    [Input('date-picker-range-page2', 'start_date'),
     Input('date-picker-range-page2', 'end_date'),
     Input('time-aggregation-page2', 'value')]
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]

@app.callback(
    Output('graph-row-page3', 'children'),
    [Input('date-picker-range-page3', 'start_date'),
     Input('date-picker-range-page3', 'end_date'),
     Input('time-aggregation-page3', 'value')]
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\Diego\AppData\Local\Temp\ipykernel_17108\2117046844.py:9: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table.Format import Group


In [2]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np
from dash_table.Format import Group
from dash import dash_table
from datetime import datetime

# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'


with open('trees.png', 'rb') as f:
    image_data_ = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para el estilo de fondo
background_style = f"data:image/jpg;base64,{image_data_}"

# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Row([
        dbc.Col([
            dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
            dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
            dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
            dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3')
        ], width=6),
        dbc.Col(html.Img(src=image_data, style={'width': '100%', 'max-width': '150px', 'float': 'right', 'margin-right': '20px'}), width=6)
    ]),
      html.Div(id='page-content')
])


# app.layout = html.Div([
#     dcc.Location(id='url', refresh=False),
#     dbc.Row([
#         dbc.Col([
#             dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
#             dbc.Button('Meteológica', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
#             dbc.Button('Predicciones Day-Ahead', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
#             dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3')
#         ], width=6),
#         dbc.Col(html.Img(src='gest.png', style={'width': '100%', 'max-width': '100px', 'float': 'right', 'margin-right': '20px'}), width=6)
#     ]),
#     html.Div(id='page-content')
# ])



# Leer la imagen y codificarla en base64


@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':
        return html.Div([
            html.H3('Meteológica', style={'background-color': '#2ECC71', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}),
            html.Div([
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=df.index.min(),
                    end_date=df.index.max(),
                    style={'display': 'inline-block', 'vertical-align': 'middle', 'marginRight': '10px'}
                ),
                dcc.Dropdown(
                    id='time-aggregation',
                    options=[
                        {'label': 'Horaria', 'value': 'H'},
                        {'label': 'Diaria', 'value': 'D'},
                        {'label': 'Semanal', 'value': 'W'},
                        {'label': 'Mensual', 'value': 'M'},
                        {'label': 'Anual', 'value': 'Y'}
                    ],
                    value='H',
                    style={'width': '150px', 'display': 'inline-block', 'vertical-align': 'middle'}
                ),
                html.Div(id='birst-level-metrics',style={'margin-bottom': '20px', 'margin-top': '30px'}),
            ]),
            dbc.Row(id='graph-row', children=[])
        ])
    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data,
                     style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('birst-level-metrics', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_birst_level_metrics(start_date, end_date):
    filtered_df = df.loc[start_date:end_date]
    last_24_records = filtered_df['Y'].tail(24)

    data = {
        "Hora": ["Predicción"],
        **{f"H{i}": [value] for i, value in enumerate(last_24_records, start=1)},
        "Media": [last_24_records.mean()]
    }
    df_table = pd.DataFrame(data)
    table = dash_table.DataTable(
        data=df_table.to_dict('records'),
        columns=[{"name": col, "id": col} for col in df_table.columns],
        style_header={
            "backgroundColor": "#4863A0",
            "color": "white",
            "fontWeight": "bold",
            "textAlign": "center"
        },
        style_cell={
            "textAlign": "center"
        }
    )
    current_date = datetime.now().date().strftime("%Y-%m-%d")
    return [html.H6(f"Predicción Day-Ahead {current_date}", className="gs-header gs-text-header padded"), table]


@app.callback(
    Output('graph-row', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation', 'value')]
)
def update_graphs(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig2 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=6),
        dbc.Col(dcc.Graph(figure=fig2), width=6)
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    [Input('date-picker-range-page2', 'start_date'),
     Input('date-picker-range-page2', 'end_date'),
     Input('time-aggregation-page2', 'value')]
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]

@app.callback(
    Output('graph-row-page3', 'children'),
    [Input('date-picker-range-page3', 'start_date'),
     Input('date-picker-range-page3', 'end_date'),
     Input('time-aggregation-page3', 'value')]
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import base64
import numpy as np
from dash_table.Format import Group
from dash import dash_table
from datetime import datetime

# Preparar los datos
date_range = pd.date_range(start='1/1/2021', periods=8760, freq='H')  # 365 days * 24 hours = 8760 hours
df = pd.DataFrame({
    "Fecha": date_range,
    "Y": [value ** 2 for value in np.random.randint(1, 11, size=len(date_range))],
    "Z": [value ** 3 for value in np.random.randint(1, 11, size=len(date_range))],
    "W": [value ** 4 for value in np.random.randint(1, 11, size=len(date_range))],
})

df.set_index('Fecha', inplace=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

with open('gest.png', 'rb') as f:
    image_encoded = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para html.Img
image_data = f'data:image/png;base64,{image_encoded}'


with open('trees.png', 'rb') as f:
    image_data2 = base64.b64encode(f.read()).decode('utf-8')

# Crea la cadena de la fuente para el estilo de fondo
background_style = f"data:image/jpg;base64,{image_data2}"

# Crear el layout de la aplicación
# Crear el layout de la aplicación
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(
        dbc.Row([
            dbc.Col([
                dbc.Button('HOME', id='home-link', href='/', external_link=True, className='m-3'),
                dbc.Button('Predicciones Day-Ahead', id='page-1-link', href='/page-1', external_link=True, className='m-3'),
                dbc.Button('Meteológica', id='page-2-link', href='/page-2', external_link=True, className='m-3'),
                dbc.Button('Precios Paises', id='page-3-link', href='/page-3', external_link=True, className='m-3')
            ], width=6),
            dbc.Col(html.Img(src=image_data, style={'width': '100%', 'max-width': '150px', 'float': 'right', 'margin-right': '20px'}), width=6)
        ]),
        style={
            'background-image': f'url({background_style})',
            'background-size': 'cover',
            'background-repeat': 'no-repeat',
            'padding': '10px'
        }
    ),

    html.Div(id='page-content')
])


# Leer la imagen y codificarla en base64


@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-1':
        return html.Div([
            html.H3('Predicción Day-Ahead', style={'background-color': '#2ECC71', 'color': 'white', 'padding': '10px', 'border-radius': '10px'}),
            html.Div(id='birst-level-metrics', style={'margin-bottom': '20px', 'margin-top': '30px'}),
            html.Div([
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=df.index.min(),
                    end_date=df.index.max(),
                    style={'display': 'inline-block', 'vertical-align': 'middle', 'marginRight': '10px', 'borderRadius': '30px'}
                ),
                dcc.Dropdown(
                    id='time-aggregation_1',
                    options=[
                        {'label': 'Horaria', 'value': 'H'},
                        {'label': 'Diaria', 'value': 'D'},
                        {'label': 'Semanal', 'value': 'W'},
                        {'label': 'Mensual', 'value': 'M'},
                        {'label': 'Anual', 'value': 'Y'}
                    ],
                    value='H',
                    style={'width': '115px', 'display': 'inline-block', 'vertical-align': 'middle', 'height': '46px'}
                ),
            ]),
            dbc.Row(id='graph-row_1', children=[]),
            html.Div([
                html.H6('Historico de Errores MAE, RMSE', className="gs-header gs-text-header padded"),
                dcc.Dropdown(
                    id='time-aggregation_2',
                    options=[
                        {'label': 'Diaria', 'value': 'D'},
                        {'label': 'Semanal', 'value': 'W'},
                        {'label': 'Mensual', 'value': 'M'},
                        {'label': 'Anual', 'value': 'Y'}
                    ],
                    value='D',
                    style={'width': '115px', 'display': 'inline-block', 'vertical-align': 'middle', 'height': '46px','margin-left': '10px'}
                ),
                html.Div(id='error-content')
            ], style={'margin-bottom': '30px'}),
            dbc.Row(id='graph-row_2', children=[]),
        ])

    elif pathname == '/page-2':
        return html.Div([
            html.H3('Página 2'),
            dcc.DatePickerRange(
                id='date-picker-range-page2',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page2',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page2', children=[])
        ])
    elif pathname == '/page-3':
        return html.Div([
            html.H3('Página 3'),
            dcc.DatePickerRange(
                id='date-picker-range-page3',
                start_date=df.index.min(),
                end_date=df.index.max()
            ),
            dcc.Dropdown(
                id='time-aggregation-page3',
                options=[
                    {'label': 'Horaria', 'value': 'H'},
                    {'label': 'Diaria', 'value': 'D'},
                    {'label': 'Semanal', 'value': 'W'},
                    {'label': 'Mensual', 'value': 'M'},
                    {'label': 'Anual', 'value': 'Y'}
                ],
                value='H'
            ),
            dbc.Row(id='graph-row-page3', children=[])
        ])
    else:
        return html.Div([
            html.H3('Bienvenido a la aplicación Dash de Gesternova.'),
            html.P('Haga clic en uno de los botones para ver las visualizaciones.'),
            html.Img(src=image_data,
                     style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'})
        ])

@app.callback(
    Output('birst-level-metrics', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_birst_level_metrics(start_date, end_date):
    filtered_df = df.loc[start_date:end_date]
    last_24_records = filtered_df['Y'].tail(24)

    data = {
        "Hora": ["Predicción"],
        **{f"H{i}": [value] for i, value in enumerate(last_24_records, start=1)},
        "Media": [last_24_records.mean()]
    }
    df_table = pd.DataFrame(data)
    table = dash_table.DataTable(
        data=df_table.to_dict('records'),
        columns=[{"name": col, "id": col} for col in df_table.columns],
        style_header={
            "backgroundColor": "#4863A0",
            "color": "white",
            "fontWeight": "bold",
            "textAlign": "center"
        },
        style_cell={
            "textAlign": "center"
        }
    )
    current_date = datetime.now().date().strftime("%Y-%m-%d")
    return [html.H6(f"Predicción Day-Ahead {current_date}", className="gs-header gs-text-header padded"), table]


@app.callback(
    Output('graph-row_1', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation_1', 'value')]
)
def update_graphs_1(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y')

    return [
        dbc.Col(dcc.Graph(figure=fig1), width=12),

    ]

@app.callback(
    Output('graph-row_2', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('time-aggregation_2', 'value')]
)

def update_graphs_2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'D':
        if time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig1 = px.line(filtered_df, x=filtered_df.index, y='Y')
    
    return [
        dbc.Col(dcc.Graph(figure=fig1), width=12),
    ]

@app.callback(
    Output('graph-row-page2', 'children'),
    [Input('date-picker-range-page2', 'start_date'),
     Input('date-picker-range-page2', 'end_date'),
     Input('time-aggregation-page2', 'value')]
)
def update_graphs_page2(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig3 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig3), width=12)
    ]




@app.callback(
    Output('graph-row-page3', 'children'),
    [Input('date-picker-range-page3', 'start_date'),
     Input('date-picker-range-page3', 'end_date'),
     Input('time-aggregation-page3', 'value')]
)
def update_graphs_page3(start_date, end_date, time_agg):
    filtered_df = df.loc[start_date:end_date]
    if time_agg != 'H':
        if time_agg == 'D':
            filtered_df = filtered_df.resample('D').mean()
        elif time_agg == 'W':
            filtered_df = filtered_df.resample('W').mean()
        elif time_agg == 'M':
            filtered_df = filtered_df.resample('M').mean()
        elif time_agg == 'Y':
            filtered_df = filtered_df.resample('Y').mean()

    fig4 = px.line(filtered_df, x=filtered_df.index, y='Y', title='Y = X^2')
    fig5 = px.line(filtered_df, x=filtered_df.index, y='Z', title='Z = X^3')
    fig6 = px.line(filtered_df, x=filtered_df.index, y='W', title='W = X^4')

    return [
        dbc.Col(dcc.Graph(figure=fig4), width=4),
        dbc.Col(dcc.Graph(figure=fig5), width=4),
        dbc.Col(dcc.Graph(figure=fig6), width=4)
    ]

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\Diego\AppData\Local\Temp\ipykernel_18148\736702088.py:9: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table.Format import Group
